# Integrazione wd_estimation.py con Dataiku DSS

Implmentazione dello script di analisi della sommersione degli edifici (`wd_estimation.py`) in un recipe Python di Dataiku DSS.

## Obiettivo
Calcolare la percentuale di sommersione degli edifici durante eventi alluvionali analizzando la profondità dell'acqua nei pixel esterni al perimetro di ciascun edificio.

## Input Dataiku
- **Dataset**: `configurazione_parametri` - Contiene i parametri di configurazione (HEIGHT_FIELD, REPROJECTION_OPTION, TARGET_EPSG, BUFFER_DISTANCE)
- **Dataset**: `configurazione_dati` - Contiene la selezione dei file di input (tipo_file, nome_file) per file vettoriali e raster specifici
- **Folder**: `minio_input` - Folder Minio con file geospaziali organizzati in sottocartelle

## Formati Supportati
### 📂 **Formati Vettoriali**:
- **Shapefile** (`.shp`) - formato standard ESRI con file accessori
- **GeoJSON** (`.geojson`, `.json`) - formato JSON geografico
- **GeoPackage** (`.gpkg`) - formato moderno OGC 
- **GeoParquet** (`.parquet`, `.geoparquet`) - formato colonnare ottimizzato per performance
- **KML** (`.kml`) - formato Google Earth
- **GML** (`.gml`) - Geography Markup Language

### 🗺️ **Formati Raster**:
- **GeoTIFF** (`.tif`, `.tiff`) - formato standard georeferenziato
- **ERDAS Imagine** (`.img`) - formato imaging professionale
- **JPEG2000** (`.jp2`) - compressione avanzata con georiferimento
- **Immagini standard** (`.png`, `.jpg`, `.jpeg`, `.bmp`, `.gif`) - con world file

## Output Dataiku  
- **Dataset**: `output_inondazioni` - DataFrame con risultati dell'analisi di sommersione
- **Folder**: `output_inondazioni` - Folder con shapefile risultanti, report statistici e file di log

## Metodologia
L'analisi utilizza la tecnica del **campionamento esterno dei pixel** per determinare la profondità dell'acqua attorno agli edifici, calcolando statistiche di sommersione basate sul rapporto tra profondità media dell'acqua e altezza dell'edificio.

### ⚠️ **Nota Tecnica - Buffer Distance**
Il parametro `BUFFER_DISTANCE` definisce la distanza (in metri) del buffer attorno agli edifici per il campionamento dei pixel:
- **Automatico** (`auto`): usa la risoluzione spaziale del raster (consigliato)
- **Manuale**: valore in metri - **IMPORTANTE**: deve essere ≥ 50% della risoluzione pixel per risultati affidabili
- **Esempio**: con risoluzione 1m, buffer < 0.5m può produrre campioni insufficienti

## 1. Setup e Configurazione

Import delle librerie necessarie e configurazione dei parametri dal dataset Dataiku.

In [ ]:
# -*- coding: utf-8 -*-
# Import librerie di base
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Import librerie geospaziali
import geopandas as gpd
import rasterio
import rasterio.mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
from shapely.geometry import mapping
from shapely.ops import unary_union
import fiona

# Import librerie di utilità
import os
import sys
import tempfile
import logging
from datetime import datetime
import pytz
import shutil
import warnings
from io import StringIO

# Configurazione per sopprimere warning di librerie geospaziali
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", message=".*Input shapes do not overlap raster.*")
warnings.filterwarnings("ignore", message=".*invalid value encountered.*")

# Classe per catturare le stampe SOLO nel log (senza output a schermo)
class LogCapture:
    def __init__(self):
        self.log_buffer = StringIO()
        self.original_stdout = sys.stdout
        
    def write(self, text):
        # Scrivi SOLO nel buffer di log - niente a schermo
        self.log_buffer.write(text)
        
    def flush(self):
        self.log_buffer.flush()
        
    def get_log_content(self):
        return self.log_buffer.getvalue()
        
    def clear_log(self):
        self.log_buffer.close()
        self.log_buffer = StringIO()
        
    def close(self):
        self.log_buffer.close()

# ATTIVA LOGGING GLOBALE PER CATTURARE TUTTE LE STAMPE
# Ripristina stdout se già attivo, poi ricrea il sistema di logging
if 'log_capture' in globals():
    sys.stdout = log_capture.original_stdout
    log_capture.close()

log_capture = LogCapture()
sys.stdout = log_capture

print("✅ Tutte le librerie importate con successo")
print("📝 Sistema di logging attivato - OUTPUT NASCOSTO")

# Ripristina stdout per mostrare solo questo messaggio di conferma
sys.stdout = log_capture.original_stdout
print("🔇 MODALITÀ SILENZIOSA ATTIVATA - Output nascosto (visibile solo nel log finale)")
sys.stdout = log_capture

In [ ]:
# Lettura dataset di configurazione
configurazione_parametri = dataiku.Dataset("configurazione_parametri")
configurazione_parametri_df = configurazione_parametri.get_dataframe()

print("=== DATASET CONFIGURAZIONE ===")
print(f"Righe nel dataset: {len(configurazione_parametri_df)}")
print(f"Colonne: {list(configurazione_parametri_df.columns)}")
print(configurazione_parametri_df.head())

# Estrazione parametri dal dataset con la struttura: (variabile, valore)
if 'variabile' in configurazione_parametri_df.columns and 'valore' in configurazione_parametri_df.columns:
    # Crea dizionario dai dati del dataset
    params_dict = dict(zip(configurazione_parametri_df['variabile'], configurazione_parametri_df['valore']))
    
    # Estrai i parametri (tutti string dal dataset)
    HEIGHT_FIELD = params_dict["HEIGHT_FIELD"]
    REPROJECTION_OPTION = int(params_dict["REPROJECTION_OPTION"])  # Converti a int
    TARGET_EPSG = params_dict["TARGET_EPSG"] 
    
    # Gestione BUFFER_DISTANCE
    buffer_val = params_dict["BUFFER_DISTANCE"]
    BUFFER_DISTANCE = None if str(buffer_val).lower() == "auto" else float(buffer_val)
    
else:
    raise ValueError("❌ Dataset deve avere colonne 'variabile' e 'valore'!")

print("\n=== PARAMETRI LETTI DAL DATASET ===")
print(f"HEIGHT_FIELD: {HEIGHT_FIELD}")
print(f"REPROJECTION_OPTION: {REPROJECTION_OPTION}")
print(f"TARGET_EPSG: {TARGET_EPSG}")
print(f"BUFFER_DISTANCE: {BUFFER_DISTANCE if BUFFER_DISTANCE is not None else 'auto (risoluzione pixel)'}")
print()

# Legge il dataset di configurazione dati per selezione file
configurazione_dati = dataiku.Dataset("configurazione_dati")
configurazione_dati_df = configurazione_dati.get_dataframe()

print("\n=== DATASET CONFIGURAZIONE DATI ===\n")
print(f"Righe nel dataset: {len(configurazione_dati_df)}")
print(f"Colonne: {list(configurazione_dati_df.columns)}")
print(configurazione_dati_df)

# Estrae i file configurati per tipo
shapefile_config = configurazione_dati_df[configurazione_dati_df['tipo_file'] == 'vettoriale']['nome_file'].iloc[0]
raster_config = configurazione_dati_df[configurazione_dati_df['tipo_file'] == 'raster']['nome_file'].iloc[0]

print(f"\n=== FILE CONFIGURATI ===\n")
print(f"Shapefile configurato: {shapefile_config}")
print(f"Raster configurato: {raster_config}")
print()

## 2. Carica Dati di Input da Dataiku

Lettura dei parametri di configurazione dal dataset e accesso ai file vettoriali e raster dal folder Minio.

In [ ]:
# Accesso al folder di input contenente i file di analisi
minio_input = dataiku.Folder("minio_input")

print("=== INFORMAZIONI FOLDER INPUT ===")
print(f"Folder minio_input: {minio_input.get_info()}")

# Elenco dei file disponibili nel folder
input_files = minio_input.list_paths_in_partition()

print(f"\nFile disponibili nel folder minio_input:")
for file_path in input_files:
    print(f"  - {file_path}")

# DEFINIZIONE FORMATI SUPPORTATI
# Formati vettoriali supportati da GeoPandas/Fiona
VECTOR_EXTENSIONS = ['.shp', '.geojson', '.json', '.gpkg', '.parquet', '.geoparquet', '.kml', '.gml']
# Formati raster supportati da Rasterio/GDAL  
RASTER_EXTENSIONS = ['.tif', '.tiff', '.img', '.jp2', '.png', '.jpg', '.jpeg', '.bmp', '.gif']

# Classificazione dei file per tipologia con supporto multi-formato
vector_files = []
raster_files = []

for file_path in input_files:
    file_lower = file_path.lower()
    
    # Controlla formati vettoriali
    if any(file_lower.endswith(ext) for ext in VECTOR_EXTENSIONS):
        vector_files.append(file_path)
    
    # Controlla formati raster
    elif any(file_lower.endswith(ext) for ext in RASTER_EXTENSIONS):
        raster_files.append(file_path)

print(f"\n=== RIEPILOGO FILE PER TIPOLOGIA ===")
print(f"File vettoriali trovati: {len(vector_files)}")
for vec in vector_files:
    file_ext = '.' + vec.split('.')[-1].upper()
    print(f"  - {vec} [{file_ext}]")

print(f"File raster trovati: {len(raster_files)}")
for ras in raster_files:
    file_ext = '.' + ras.split('.')[-1].upper()
    print(f"  - {ras} [{file_ext}]")

# Verifica presenza dei file necessari
if len(vector_files) == 0:
    supported_vec = ', '.join(VECTOR_EXTENSIONS)
    raise ValueError(f"❌ Nessun file vettoriale trovato nel folder minio_input!\n"
                    f"Formati supportati: {supported_vec}")
                    
if len(raster_files) == 0:
    supported_ras = ', '.join(RASTER_EXTENSIONS)
    raise ValueError(f"❌ Nessun file raster trovato nel folder minio_input!\n"
                    f"Formati supportati: {supported_ras}")

print(f"\n📋 Formati vettoriali supportati: {', '.join(VECTOR_EXTENSIONS)}")
print(f"📋 Formati raster supportati: {', '.join(RASTER_EXTENSIONS[:8])}... (+{len(RASTER_EXTENSIONS)-8} altri)")

# SELEZIONE INTELLIGENTE BASATA SU CONFIGURAZIONE
def find_configured_file(file_list, configured_path, file_type):
    """Trova il file che corrisponde alla configurazione con percorso completo"""
    # Estrai il nome del file dalla configurazione (dopo l'ultimo /)
    configured_filename = configured_path.split('/')[-1]
    
    # Cerca corrispondenza esatta per nome file
    exact_matches = [f for f in file_list if configured_filename in f]
    if exact_matches:
        print(f"✓ Trovato {file_type} configurato '{configured_filename}': {exact_matches[0]}")
        return exact_matches[0]
    
    # Cerca corrispondenza parziale nel percorso completo
    path_matches = [f for f in file_list if configured_path.replace('/', '\\') in f or configured_path.replace('\\', '/') in f]
    if path_matches:
        print(f"✓ Trovato {file_type} con percorso '{configured_path}': {path_matches[0]}")
        return path_matches[0]
    
    # Se non trova corrispondenza, usa il primo e avvisa
    print(f"⚠️  {file_type} configurato '{configured_path}' non trovato, uso il primo disponibile: {file_list[0]}")
    print(f"   - File cercato: {configured_filename}")
    print(f"   - Percorso cercato: {configured_path}")
    return file_list[0]

print(f"\n=== SELEZIONE BASATA SU CONFIGURAZIONE ===")
print(f"📁 File vettoriale configurato: {shapefile_config}")
print(f"📁 File raster configurato: {raster_config}")

# Seleziona i file basandosi sulla configurazione (ora con nomi generici)
vector_file = find_configured_file(vector_files, shapefile_config, "file vettoriale")
raster_file = find_configured_file(raster_files, raster_config, "file raster")

print(f"\n=== FILE SELEZIONATI PER L'ANALISI ===\n")
print(f"📄 File vettoriale: {vector_file}")
print(f"🗺️  File raster: {raster_file}")

# Mostra file alternativi disponibili
if len(vector_files) > 1:
    print(f"\n📋 Altri file vettoriali disponibili:")
    for vec in vector_files:
        if vec != vector_file:
            file_ext = '.' + vec.split('.')[-1].upper()
            print(f"   - {vec} [{file_ext}]")

if len(raster_files) > 1:
    print(f"\n📋 Altri file raster disponibili:")
    for ras in raster_files:
        if ras != raster_file:
            file_ext = '.' + ras.split('.')[-1].upper()
            print(f"   - {ras} [{file_ext}]")

print(f"\n💡 Per cambiare selezione, modifica il dataset 'configurazione_dati'")

In [ ]:
# Download dei file dal folder di input verso directory temporanea locale
temp_dir = tempfile.mkdtemp()
print(f"Directory temporanea creata: {temp_dir}")

# Definizione percorsi locali per i file selezionati
vector_local_path = os.path.join(temp_dir, os.path.basename(vector_file))
raster_local_path = os.path.join(temp_dir, os.path.basename(raster_file))

# Scaricamento file vettoriale principale
print(f"Download in corso: {vector_file}")
with open(vector_local_path, 'wb') as f:
    f.write(minio_input.get_download_stream(vector_file).read())

# Scaricamento file raster
print(f"Download in corso: {raster_file}")
with open(raster_local_path, 'wb') as f:
    f.write(minio_input.get_download_stream(raster_file).read())

# Scaricamento file accessori per shapefile (se il file vettoriale è uno shapefile)
if vector_file.lower().endswith('.shp'):
    print(f"📁 Rilevato Shapefile - scaricamento file accessori...")
    shapefile_extensions = ['.dbf', '.shx', '.prj', '.qix', '.xml', '.cpg', '.sbx', '.sbn']
    
    for ext in shapefile_extensions:
        aux_file = vector_file.replace('.shp', ext).replace('.SHP', ext)
        if aux_file in input_files:
            aux_local_path = os.path.join(temp_dir, os.path.basename(aux_file))
            print(f"Download file accessorio: {aux_file}")
            with open(aux_local_path, 'wb') as f:
                f.write(minio_input.get_download_stream(aux_file).read())
            print(f"Completato: {os.path.basename(aux_file)}")

# Scaricamento file accessori per altri formati (se presenti)
elif vector_file.lower().endswith(('.gpkg', '.gdb')):
    print(f"📁 File vettoriale database rilevato - formato autocontenuto")
    
elif vector_file.lower().endswith(('.parquet', '.geoparquet')):
    print(f"📁 File GeoParquet rilevato - formato colonnare ottimizzato")
    
else:
    vector_ext = vector_file.split('.')[-1].upper()
    print(f"📁 File vettoriale {vector_ext} - formato autocontenuto")

# Informazioni sui file raster
raster_ext = raster_file.split('.')[-1].upper() 
print(f"📁 File raster {raster_ext} scaricato")

print(f"\n=== DOWNLOAD COMPLETATO ===")
print(f"File vettoriale: {vector_local_path}")
print(f"File raster: {raster_local_path}")
print(f"Directory di lavoro: {temp_dir}")
print(f"Formato vettoriale: {vector_file.split('.')[-1].upper()}")
print(f"Formato raster: {raster_file.split('.')[-1].upper()}")

In [ ]:
# Carica dati vettoriali e raster con geopandas e rasterio
vector = gpd.read_file(vector_local_path)
raster = rasterio.open(raster_local_path)

print("=== DATI CARICATI ===")
print(f"Edifici nel vettoriale: {len(vector)}")
print(f"Dimensioni raster: {raster.width} x {raster.height}")
print(f"CRS vettoriale: {vector.crs}")
print(f"CRS raster: {raster.crs}")

# Controlla i campi disponibili nel vettoriale
print(f"\nCampi disponibili nel vettoriale:")
print(list(vector.columns))

# Verifica che il campo altezza sia presente
if HEIGHT_FIELD not in vector.columns:
    raise ValueError(f"Campo altezza '{HEIGHT_FIELD}' non trovato nel vettoriale! Campi disponibili: {list(vector.columns)}")
    
print(f"\n✓ Campo altezza '{HEIGHT_FIELD}' trovato nel vettoriale")

# Rilevamento campo FID (case-insensitive)
FID_FIELD = None
fid_value_source = None
for col in vector.columns:
    if col.upper() == 'FID':
        FID_FIELD = col
        fid_value_source = 'input'  # Eredita dall'input
        break

if FID_FIELD:
    print(f"✓ Campo FID trovato nell'input: '{FID_FIELD}' - sarà ereditato")
else:
    print("ℹ️  Campo FID non presente nell'input - sarà generato automaticamente")
    fid_value_source = 'generated'  # Genera automaticamente

## 3. Allineamento Sistemi di Riferimento

Controllo della compatibilità CRS tra dati vettoriali e raster e implementazione della logica di riproiezione.

In [ ]:
# Controllo CRS
vector_crs = vector.crs
raster_crs = raster.crs

if vector_crs != raster_crs:
    print(f"⚠️  ATTENZIONE: I sistemi di riferimento non coincidono!")
    print(f"CRS vettoriale: {vector_crs}")
    print(f"CRS raster: {raster_crs}")
    print(f"Applicando opzione di riproiezione: {REPROJECTION_OPTION}")
    
    try:
        if REPROJECTION_OPTION == 1:
            # Riproietta vettoriale nel CRS del raster
            target_crs = raster_crs
            print(f"Riproiettando il vettoriale in {target_crs}...")
            vector = vector.to_crs(target_crs)
            print("✓ Vettoriale riproiettato.")
            
        elif REPROJECTION_OPTION == 2:
            # Riproietta raster nel CRS del vettoriale
            target_crs = vector_crs
            print(f"Riproiettando il raster in {target_crs}...")
            # Crea file temporaneo per raster riproiettato
            temp_raster = tempfile.NamedTemporaryFile(suffix='.tif', delete=False)
            temp_raster_path = temp_raster.name
            temp_raster.close()
            
            # Calcola trasformazione
            transform, width, height = calculate_default_transform(
                raster.crs, target_crs, raster.width, raster.height, *raster.bounds)
            
            # Parametri per il nuovo raster
            kwargs = raster.meta.copy()
            kwargs.update({
                'crs': target_crs,
                'transform': transform,
                'width': width,
                'height': height
            })
            
            # Esegui riproiezione
            with rasterio.open(temp_raster_path, 'w', **kwargs) as dst:
                for i in range(1, raster.count + 1):
                    reproject(
                        source=rasterio.band(raster, i),
                        destination=rasterio.band(dst, i),
                        src_transform=raster.transform,
                        src_crs=raster.crs,
                        dst_transform=transform,
                        dst_crs=target_crs,
                        resampling=Resampling.bilinear)
            
            # Chiudi raster originale e apri quello riproiettato
            raster.close()
            raster = rasterio.open(temp_raster_path)
            print("✓ Raster riproiettato.")
            
        elif REPROJECTION_OPTION == 3:
            # Riproietta entrambi nel CRS specificato
            target_crs = f"EPSG:{TARGET_EPSG}"
            print(f"Riproiettando entrambi in {target_crs}...")
            
            # Riproietta vettoriale
            vector = vector.to_crs(target_crs)
            print("✓ Vettoriale riproiettato.")
            
            # Riproietta raster (stesso codice dell'opzione 2)
            temp_raster = tempfile.NamedTemporaryFile(suffix='.tif', delete=False)
            temp_raster_path = temp_raster.name
            temp_raster.close()
            
            transform, width, height = calculate_default_transform(
                raster.crs, target_crs, raster.width, raster.height, *raster.bounds)
            
            kwargs = raster.meta.copy()
            kwargs.update({
                'crs': target_crs,
                'transform': transform,
                'width': width,
                'height': height
            })
            
            with rasterio.open(temp_raster_path, 'w', **kwargs) as dst:
                for i in range(1, raster.count + 1):
                    reproject(
                        source=rasterio.band(raster, i),
                        destination=rasterio.band(dst, i),
                        src_transform=raster.transform,
                        src_crs=raster.crs,
                        dst_transform=transform,
                        dst_crs=target_crs,
                        resampling=Resampling.bilinear)
            
            raster.close()
            raster = rasterio.open(temp_raster_path)
            print("✓ Raster riproiettato.")
            
        else:
            raise ValueError(f"Opzione di riproiezione non valida: {REPROJECTION_OPTION}")
            
    except Exception as e:
        raise Exception(f"Errore durante la riproiezione: {e}")

else:
    print("✓ Sistemi di riferimento già compatibili - nessuna riproiezione necessaria")

## 4. Funzioni di Analisi della Profondità dell'Acqua

Implementazione della funzione `get_external_pixels()` per estrarre i valori di profondità dell'acqua dai pixel immediatamente esterni al perimetro degli edifici.

In [ ]:
def get_external_pixels(geom, raster, buffer_distance=None):
    """
    Estrae i valori dei pixel immediatamente esterni al perimetro del poligono
    
    Parametri:
    - geom: geometria del poligono (edificio)  
    - raster: rasterio dataset con profondità acqua
    - buffer_distance: distanza buffer in metri (None = automatico = risoluzione pixel)
    
    Ritorna:
    - numpy array con valori di profondità validi
    """
    try:
        # Disabilita temporaneamente tutti i warning per evitare messaggi di sovrapposizione
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            
            # Se non specificato, usa la risoluzione del raster come buffer
            if buffer_distance is None:
                buffer_distance = abs(raster.transform[0])  # risoluzione pixel
            
            # Crea buffer esterno molto piccolo
            external_buffer = geom.buffer(buffer_distance)
            
            # Crea anello: buffer esterno - poligono originale
            ring = external_buffer.difference(geom)
            
            # Estrai valori raster dall'anello - questa chiamata può generare il warning
            out_image, out_transform = rasterio.mask.mask(raster, [mapping(ring)], crop=True, filled=True)
            data = out_image[0]
            
            # Escludi nodata
            valid_data = data[data != raster.nodata]
            
            return valid_data
        
    except Exception as e:
        # Gestione silenziosa degli errori comuni (es. nessuna sovrapposizione)
        # Gli errori saranno tracciati nel conteggio generale
        return np.array([])

print("✓ Funzione get_external_pixels() definita con soppressione warning")

## 5. Calcolo della Sommersione degli Edifici

Elaborazione di ogni edificio per calcolare area, volume e statistiche di sommersione con tracking del progresso.

In [ ]:
# Prepara lista risultati e contatori dettagliati
results = []
processed_count = 0
not_processed_count = 0
no_overlap_count = 0  # Edifici senza sovrapposizione con raster
zero_height_count = 0  # Edifici con altezza zero/negativa
other_errors_count = 0  # Altri errori

print(f"🚀 Inizio elaborazione di {len(vector)} edifici...")
print(f"Buffer distance: {BUFFER_DISTANCE} (None = automatico)")

for idx, row in vector.iterrows():
    geom = row.geometry
    a_base = geom.area  # Calcola area dalla geometria
    h_uvl = row[HEIGHT_FIELD]  # Legge altezza dal campo parametrizzato
    vol = a_base * h_uvl
    
    # Controlla validità altezza prima dell'estrazione
    if h_uvl <= 0:
        zero_height_count += 1
        depth_mean = 0.0
        depth_min = 0.0
        depth_max = 0.0
        perc_submerged = 0.0
        not_processed_count += 1
    else:
        # Estrai valori esterni al perimetro
        external_values = get_external_pixels(geom, raster, BUFFER_DISTANCE)
        
        if external_values.size > 0:
            # Calcola statistiche di sommersione
            depth_mean = np.mean(external_values)
            depth_min = np.min(external_values)
            depth_max = np.max(external_values)
            
            # Calcola percentuale di sommersione basata sulla quota media
            perc_submerged = min((depth_mean / h_uvl) * 100, 100.0)
            
            processed_count += 1
        else:
            # Nessun dato valido - probabilmente nessuna sovrapposizione
            no_overlap_count += 1
            depth_mean = 0.0
            depth_min = 0.0
            depth_max = 0.0
            perc_submerged = 0.0
            not_processed_count += 1
    
    # Gestione campo FID
    if fid_value_source == 'input':
        fid_value = row[FID_FIELD]  # Eredita dall'input
    else:
        fid_value = idx + 1  # Genera FID sequenziale (1-based)
    
    # Crea record risultato - include FID
    result_record = {
        'FID': fid_value,
        'A_BASE': round(a_base, 2),
        HEIGHT_FIELD: round(h_uvl, 2),
        'VOL': round(vol, 2),
        'DEPTH_MEAN': round(depth_mean, 2),
        'DEPTH_MIN': round(depth_min, 2),
        'DEPTH_MAX': round(depth_max, 2),
        'PERC_SUBM': round(perc_submerged, 2),
        'geometry': geom
    }
    
    results.append(result_record)
    
    # Progress indicator
    if (idx + 1) % 100 == 0:
        print(f"📊 Elaborati {idx + 1}/{len(vector)} edifici...")

print(f"✅ Elaborazione completata!")
print(f"Edifici processati con successo: {processed_count}")
print(f"Edifici non processati: {not_processed_count}")
if not_processed_count > 0:
    print(f"  └─ Senza sovrapposizione con raster: {no_overlap_count}")
    print(f"  └─ Altezza zero/negativa: {zero_height_count}")
    if other_errors_count > 0:
        print(f"  └─ Altri errori: {other_errors_count}")

## 6. Preparazione Output

Creazione del GeoDataFrame di output con i risultati dell'analisi di sommersione.

In [ ]:
# Crea GeoDataFrame di output
out_gdf = gpd.GeoDataFrame(results, crs=vector.crs)
total_buildings = len(vector)  # Variabile necessaria per il report

print(f"✓ GeoDataFrame di output pronto: {len(out_gdf)} record con campi analisi aggiunti")

## 7. Generazione Output

Creazione del DataFrame di output con schema appropriato e scrittura nel dataset Dataiku di destinazione.

In [ ]:
# Converti GeoDataFrame in DataFrame standard per Dataiku
output_inondazioni_df = pd.DataFrame(out_gdf.drop(columns='geometry'))

# Aggiungi colonna WKT come prima colonna per il CSV
output_inondazioni_df.insert(0, 'geometry_wkt', out_gdf['geometry'].apply(lambda x: x.wkt))

# Assicura che FID sia la seconda colonna
if 'FID' in output_inondazioni_df.columns:
    fid_col = output_inondazioni_df.pop('FID')
    output_inondazioni_df.insert(1, 'FID', fid_col)

print(f"📋 DataFrame per output Dataiku:")
print(f"   Righe: {len(output_inondazioni_df)}")
print(f"   Colonne: {len(output_inondazioni_df.columns)}")
print(f"   Colonne: {list(output_inondazioni_df.columns)}")

# Mostra esempio primi record
print(f"\n📊 Esempio primi 3 record:")
print(output_inondazioni_df.head(3))

In [ ]:
# Salvataggio risultati come CSV
print("📝 Salvataggio file CSV...")

italian_tz = pytz.timezone('Europe/Rome')
timestamp = datetime.now(italian_tz).strftime("%Y%m%d_%H%M%S") 
csv_filename = f"risultati_inondazioni_{timestamp}.csv"

try:
    output_folder = dataiku.Folder("output_inondazioni")
    csv_content = output_inondazioni_df.to_csv(index=False)
    
    from io import StringIO
    csv_stream = StringIO(csv_content)
    output_folder.upload_stream(csv_filename, csv_stream.getvalue().encode('utf-8'))
    
    print(f"✅ File CSV salvato: {csv_filename}")
    print(f"✅ {len(output_inondazioni_df)} record salvati")
    
except Exception as e:
    local_csv = f"C:\\temp\\{csv_filename}"
    output_inondazioni_df.to_csv(local_csv, index=False)
    print(f"✅ File salvato in locale: {local_csv}")
    print(f"✅ {len(output_inondazioni_df)} record salvati")

print("✅ ELABORAZIONE COMPLETATA")
print(f"✅ Dataset 'output_inondazioni' scritto con {len(output_inondazioni_df)} record")
print(f"✅ Analisi di sommersione completata per {processed_count}/{total_buildings} edifici")

## 8. Salvataggio File Fisici

Salvataggio di shapefile, report statistico e upload nel folder Dataiku di output.

In [ ]:
# Configurazione folder di output per il salvataggio dei risultati
output_folder = dataiku.Folder("output_inondazioni")

# Creazione directory temporanea per i file di output
output_temp_dir = tempfile.mkdtemp()
print(f"Directory temporanea per output: {output_temp_dir}")

# Generazione nome base per i file di output con timestamp
italian_tz = pytz.timezone('Europe/Rome')
timestamp = datetime.now(italian_tz).strftime("%Y%m%d_%H%M%S")
output_base_name = f"wd_analysis_{timestamp}"

# Definizione percorsi dei file di output
shapefile_path = os.path.join(output_temp_dir, f"{output_base_name}.shp")
report_path = os.path.join(output_temp_dir, f"{output_base_name}_report.txt")
log_path = os.path.join(output_temp_dir, f"{output_base_name}.log")

print(f"File di output programmati:")
print(f"   Shapefile: {os.path.basename(shapefile_path)}")
print(f"   Report statistico: {os.path.basename(report_path)}")
print(f"   Log elaborazione: {os.path.basename(log_path)}")

In [ ]:
# Salva vettoriale con schema definito (include FID)
schema = {
    'geometry': 'Polygon',
    'properties': {
        'FID': 'int:10',             # Campo identificativo
        'A_BASE': 'float:10.2',      # 10 cifre totali, 2 decimali
        HEIGHT_FIELD: 'float:8.2',   # 8 cifre totali, 2 decimali  
        'VOL': 'float:12.2',         # 12 cifre totali, 2 decimali
        'DEPTH_MEAN': 'float:8.2',   # 8 cifre totali, 2 decimali
        'DEPTH_MIN': 'float:8.2',    # 8 cifre totali, 2 decimali
        'DEPTH_MAX': 'float:8.2',    # 8 cifre totali, 2 decimali
        'PERC_SUBM': 'float:6.2'     # 6 cifre totali, 2 decimali
    }
}

print("💾 Salvataggio vettoriale...")
with fiona.open(shapefile_path, 'w', driver='ESRI Shapefile', crs=out_gdf.crs, schema=schema) as f:
    for idx, row in out_gdf.iterrows():
        feature = {
            'geometry': mapping(row.geometry),
            'properties': {
                'FID': int(row['FID']),
                'A_BASE': float(row['A_BASE']),
                HEIGHT_FIELD: float(row[HEIGHT_FIELD]),
                'VOL': float(row['VOL']),
                'DEPTH_MEAN': float(row['DEPTH_MEAN']),
                'DEPTH_MIN': float(row['DEPTH_MIN']),
                'DEPTH_MAX': float(row['DEPTH_MAX']),
                'PERC_SUBM': float(row['PERC_SUBM'])
            }
        }
        f.write(feature)

In [ ]:
# Crea report statistico dettagliato
print("📊 Generazione report statistico...")

with open(report_path, 'w', encoding='utf-8') as f:
    f.write("=== REPORT ANALISI SOMMERSIONE EDIFICI ===\n\n")
    f.write(f"Data elaborazione: {datetime.now(pytz.timezone('Europe/Rome')).strftime('%Y-%m-%d %H:%M:%S')} (Ora italiana)\n")
    f.write(f"Versione script: dataiku_integration.ipynb\n")
    f.write(f"Campo altezza utilizzato: {HEIGHT_FIELD}\n")
    f.write(f"Opzione riproiezione: {REPROJECTION_OPTION} ")
    if REPROJECTION_OPTION == 1:
        f.write("(riproietta vettoriale)\n")
    elif REPROJECTION_OPTION == 2:
        f.write("(riproietta raster)\n")
    else:
        f.write(f"(riproietta entrambi in {TARGET_EPSG})\n")
    f.write(f"Buffer distance: {BUFFER_DISTANCE}\n\n")
    
    f.write("=== FILE DI INPUT/OUTPUT ===\n")
    f.write(f"File vettoriale: {vector_file}\n")
    f.write(f"File raster: {raster_file}\n")
    f.write(f"File output shapefile: {os.path.basename(shapefile_path)}\n")
    f.write(f"File output report: {os.path.basename(report_path)}\n\n")
    
    f.write("=== SISTEMI DI RIFERIMENTO ===\n")
    f.write(f"CRS vettoriale originale: {vector_crs}\n")
    f.write(f"CRS raster: {raster_crs}\n")
    if vector_crs != raster_crs:
        f.write("NOTA: Sistemi di riferimento diversi - applicata riproiezione automatica\n")
    else:
        f.write("NOTA: Sistemi di riferimento coincidenti - nessuna riproiezione necessaria\n")
    f.write("\n")
    
    f.write("=== RIEPILOGO ELABORAZIONE ===\n")
    f.write(f"Edifici totali nel vettoriale: {total_buildings}\n")
    f.write(f"Edifici processati con successo: {processed_count} ({processed_count/total_buildings*100:.1f}%)\n")
    f.write(f"Edifici non processati: {not_processed_count} ({not_processed_count/total_buildings*100:.1f}%)\n")
    f.write(f"  - Cause: senza sovrapposizione con raster, altezza zero/negativa, errori geometrici\n\n")
    
    f.write("=== METODOLOGIA ===\n")
    f.write("L'analisi calcola la sommersione degli edifici campionando i valori di profondità\n")
    f.write("dell'acqua nei pixel esterni al perimetro di ciascun edificio.\n")
    f.write("La percentuale di sommersione è calcolata come: (profondità_media / altezza_edificio) × 100\n")
    f.write("I valori sono limitati al 100% per edifici completamente sommersi.\n\n")
    
    # Statistiche dettagliate se ci sono edifici processati
    if processed_count > 0:
        processed_data = out_gdf[out_gdf['DEPTH_MEAN'] > 0]
        
        if len(processed_data) > 0:
            f.write(f"Edifici con sommersione rilevata: {len(processed_data)} ({len(processed_data)/total_buildings*100:.1f}%)\n\n")
            
            # Statistiche profondità
            mean_depth = processed_data['DEPTH_MEAN'].mean()  
            max_depth = processed_data['DEPTH_MAX'].max()
            min_depth = processed_data['DEPTH_MIN'].min()
            
            f.write("=== PROFONDITÀ ACQUA ===\n")
            f.write(f"Profondità media: {mean_depth:.2f} m (range: {min_depth:.2f} - {max_depth:.2f} m)\n\n")
            
            # Classificazione edifici
            edifici_bassi = len(processed_data[processed_data['PERC_SUBM'] < 25])
            edifici_medi = len(processed_data[(processed_data['PERC_SUBM'] >= 25) & (processed_data['PERC_SUBM'] < 75)])
            edifici_alti = len(processed_data[processed_data['PERC_SUBM'] >= 75])
            
            f.write("=== CLASSIFICAZIONE EDIFICI PER LIVELLO SOMMERSIONE ===\n")
            f.write(f"Sommersione bassa (<25%): {edifici_bassi} edifici ({edifici_bassi/len(processed_data)*100:.1f}%)\n")
            f.write(f"Sommersione media (25-75%): {edifici_medi} edifici ({edifici_medi/len(processed_data)*100:.1f}%)\n")
            f.write(f"Sommersione alta (≥75%): {edifici_alti} edifici ({edifici_alti/len(processed_data)*100:.1f}%)\n\n")
    
    f.write("=== CAMPI OUTPUT VETTORIALE ===\n")
    fid_source_text = "ereditato dall'input" if fid_value_source == 'input' else 'generato automaticamente'
    f.write(f"FID: Identificativo univoco edificio ({fid_source_text})\n")
    f.write("DEPTH_MEAN: Profondità media dell'acqua attorno all'edificio (m)\n")
    f.write("DEPTH_MAX: Profondità massima dell'acqua attorno all'edificio (m)\n")
    f.write("DEPTH_MIN: Profondità minima dell'acqua attorno all'edificio (m)\n")
    f.write("PERC_SUBM: Percentuale di sommersione dell'edificio (%)\n")
    f.write(f"{HEIGHT_FIELD}: Altezza dell'edificio utilizzata nel calcolo (m)\n")
    f.write("A_BASE: Area della base dell'edificio (m²)\n")
    f.write("VOL: Volume dell'edificio (m³)\n")

print(f"✅ Report salvato: {os.path.basename(report_path)}")

In [ ]:
# Crea il file di log con TUTTE le stampe catturate
print("📝 Generazione file di log completo...")
with open(log_path, 'w', encoding='utf-8') as f:
    f.write(f"=== LOG ELABORAZIONE ANALISI SOMMERSIONE EDIFICI ===\n\n")
    f.write(f"Data elaborazione: {datetime.now(pytz.timezone('Europe/Rome')).strftime('%Y-%m-%d %H:%M:%S')} (Ora italiana)\n")
    f.write(f"Versione script: dataiku_integration.ipynb\n")
    f.write(f"Parametri utilizzati:\n")
    f.write(f"  - HEIGHT_FIELD: {HEIGHT_FIELD}\n")
    f.write(f"  - REPROJECTION_OPTION: {REPROJECTION_OPTION}\n")
    f.write(f"  - TARGET_EPSG: {TARGET_EPSG}\n")
    f.write(f"  - BUFFER_DISTANCE: {BUFFER_DISTANCE}\n")
    f.write(f"\n=== TRANSCRIPT ESECUZIONE ===\n\n")
    f.write(log_capture.get_log_content())
    f.write(f"\n=== FINE LOG ===\n")

# Upload dei file di output nel folder Dataiku
print("📤 Upload file nel folder Dataiku di output...")

# Lista di tutti i file da caricare (shapefile + accessori + report + log)
files_to_upload = []

# Shapefile principale
files_to_upload.append(shapefile_path)
# File accessori shapefile
for ext in ['.dbf', '.shx', '.prj', '.cpg']:
    aux_file = shapefile_path.replace('.shp', ext)
    if os.path.exists(aux_file):
        files_to_upload.append(aux_file)

# Report
files_to_upload.append(report_path)

# File di log
files_to_upload.append(log_path)

# Upload dei file
uploaded_files = []
for file_path in files_to_upload:
    if os.path.exists(file_path):
        file_name = os.path.basename(file_path)
        try:
            with open(file_path, 'rb') as f:
                output_folder.upload_stream(file_name, f)
            uploaded_files.append(file_name)
        except Exception as e:
            pass  # Gestione silenziosa degli errori

# Aggiorna il log con le informazioni di upload
with open(log_path, 'a', encoding='utf-8') as f:
    f.write(f"\n=== OPERAZIONI DI UPLOAD ===\n\n")
    f.write(f"📤 Upload completato nel folder 'minio/output'\n")
    f.write(f"📁 File caricati: {len(uploaded_files)}\n")
    for filename in uploaded_files:
        f.write(f"   ✅ {filename}\n")
    f.write(f"\n🎉 SALVATAGGIO COMPLETATO!\n")

# Re-upload del log aggiornato
try:
    with open(log_path, 'rb') as f:
        output_folder.upload_stream(os.path.basename(log_path), f)
except:
    pass

# Pulizia directory temporanea output  
try:
    shutil.rmtree(output_temp_dir)
except Exception as e:
    pass

In [ ]:
# Pulizia risorse e file temporanei
try:
    raster.close()
    if 'temp_raster_path' in locals():
        try:
            os.unlink(temp_raster_path)
        except:
            pass

    # Pulizia directory temporanea
    try:
        shutil.rmtree(temp_dir)
    except:
        pass
    
except Exception as e:
    pass

# Ripristina stdout originale e chiudi il sistema di logging
sys.stdout = log_capture.original_stdout
log_capture.close()